In [1]:
### Preface
import javabridge
import bioformats as bf
javabridge.start_vm(class_path=bf.JARS)
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sys import path as syspath
syspath.append("./functions/")
from physio_def_1 import getApparentFreq, importFrames, getTimes

from plotFirst_1 import plotImage

from shutil import move as movecarefully

import os

In [2]:
def do_single_Series(iSeries):
    from warnings import warn
    try:
        im = xml.image(iSeries)
        Name = im.Name

        if Nimages==1:
            npzFile   = filename.replace(extension,".npz")
            imageFile = filename.replace(extension,".png")
            dimFile   = filename.replace(extension,".txt")
        else:
            npzFile   = filename.replace(extension,f"_{iSeries}_{Name}.npz")
            imageFile = filename.replace(extension,f"_{iSeries}_{Name}.png")
            dimFile   = filename.replace(extension,f"_{iSeries}_{Name}.txt")

        npzFile   = os.path.join(newdir,npzFile   )
        imageFile = os.path.join(newdir,imageFile )
        dimFile   = os.path.join(newdir,dimFile   )

        dimensions = dict(zip("TXY",(getattr(im.Pixels, "Size"+dim) for dim in "TXY")))

        if dimensions["T"]>1:
            dimensions['freq'] = getApparentFreq(iSeries,xml)

        howManyFirstFrames = min(10,dimensions["T"])
        firstFrames = importFrames(rdr,idx=iSeries, which=(howManyFirstFrames,))

        extraDims = len(firstFrames.shape)>3
        if extraDims:
        #         print("There are more dimensions here than expected.\nI'll remedy this for now, but be shure to keep this in mind.")
            retainDims = np.where(firstFrames.std(axis=tuple(range(len(firstFrames.shape)-1)))>0)[0]
            assert len(retainDims)==1
            retainDims = retainDims[0]
            firstFrames = firstFrames.T[retainDims].T
        else:
            retainDims = None

        meanFirstFrames = firstFrames.mean(axis=0)
        #     stdFirstFrames = firstFrames.std(axis=0)

        pxSize = im.Pixels.get_PhysicalSizeX()
        pxUnit = im.Pixels.get_PhysicalSizeXUnit()
        text = "\n".join([" %s:%i"%(c,dimensions[c])  for c in "XYT"])
        if dimensions["T"]>1:
            text += "\n f:%.1f Hz"%dimensions['freq']
        dimensions["pxSize"] = pxSize
        dimensions["pxUnit"] = pxUnit
        pd.DataFrame([dimensions]).to_csv(dimFile, index=False)
        
        try:
            plotImage(meanFirstFrames.T,pxSize=pxSize,pxUnit=pxUnit,
                      savePath=imageFile,
                      addInfo=text)
            plt.close()
        except:
            pass

        time = getTimes(xml_=xml,idx_=iSeries)

        mintype = pd.to_numeric(firstFrames.flatten(),downcast="unsigned").dtype
        allimage = importFrames(
            rdr,
            idx=iSeries,
            dtype=mintype,
            which=(dimensions['T'],None,None,1)[:3+int(extraDims)]
        #     which=(None,None,None)+tuple([retainDims][:int(extraDims)])
        )
        np.savez_compressed(npzFile,time=time,data=allimage)
        return 0
    except:
        warn(f"something was off doing {iSeries} in file {pathToRecord}.\nI'll continue, but with a note in the folder.")
        with open(os.path.join(newdir,filename.replace(extension,".err")),"a") as fw:
            fw.write(f"error for series number {iSeries}\n")
        return 1

In [3]:
def prepareOverview(directory):    
    overview = []
    for txtname in os.listdir(directory)[::-1]:
        if not txtname.endswith("txt"):
            continue
        try:
            sr = pd.read_csv(os.path.join(directory,txtname)).loc[0]
            sr = sr.append(pd.Series({
                "filename"    :    txtname.rstrip(".txt"),
                "iSeries"     :int(txtname.split("_")[-2]),
                "Series_Name" :    txtname.split("_")[-1],
            }))
        except:
            sr = pd.Series({"filename":txtname})
        overview += [sr]

    overview = pd.DataFrame(overview)
    overview.set_index("iSeries",inplace=True)
    overview.sort_index(inplace=True)

    overview.to_csv(os.path.join(directory,"overview.csv"))

# If you want to parse a single record

In [4]:
pathToRecord = '/Volumes/physio/team/slakrupnik/project/experiments/MB/Cytokines/cytokines-2.lif'

In [5]:
md = bf.get_omexml_metadata(pathToRecord)
xml = bf.OMEXML(md)
Nimages = xml.get_image_count()
rdr = bf.ImageReader(pathToRecord, perform_init=True)
pathToDir, filename = os.path.split(pathToRecord)
extension = "."+filename.split(".")[-1]
newdir = os.path.join(pathToDir,filename.rstrip(extension))

if not os.path.isdir(newdir):
    os.makedirs(newdir)

errd = 0
for iSeries in range(Nimages):
    errd += do_single_Series(iSeries)

if errd==0:
    movecarefully(pathToRecord,os.path.join(newdir,filename+"_done"))

if Nimages>1:
    prepareOverview(newdir)

In [9]:
# xml.to_xml()

# If you want to parse all records within a directory

In [7]:
pathToFolder = '/Volumes/physio/team/slakrupnik/project/experiments/MB/Cytokines/'

In [ ]:
for filename in os.listdir(pathToFolder):
    pathToRecord = os.path.join(pathToFolder,filename)
    try:
        md = bf.get_omexml_metadata(pathToRecord)
    except:
        # if it is something else
        continue
    xml = bf.OMEXML(md)
    Nimages = xml.get_image_count()
    rdr = bf.ImageReader(pathToRecord, perform_init=True)
    pathToDir, filename = os.path.split(pathToRecord)
    extension = "."+filename.split(".")[-1]
    newdir = os.path.join(pathToDir,filename.rstrip(extension))

    if not os.path.isdir(newdir):
        os.makedirs(newdir)

    errd = 0
    for iSeries in range(Nimages):
        errd += do_single_Series(iSeries)

    if errd==0:
        movecarefully(pathToRecord,os.path.join(newdir,filename+"_done"))

    if Nimages>1:
        prepareOverview(newdir)